# Tutorial 2: Running Aimsun Simulations

This tutorial walks through the process of running non-RL traffic simulations in Flow with Aimsun. This tutorial is a near direct copy of the `tutorial01_sumo.ipynb` since some modifications to describe Aimsun-specific features. This tutorial, however, is self-sufficient for readers who have not gone through the previous tutorial.

*If, instead of running a Flow-created network in Aimsun, you wish to run an already existing Aimsun template using Flow, please refer to the tutorial on network templates (`tutorial08_network_templates.ipynb`).*

Simulations of this form act as non-autonomous baselines and depict the behavior of human dynamics on a network. Similar simulations may also be used to evaluate the performance of hand-designed controllers on a network. This tutorial focuses primarily on the former use case, while an example of the latter may be found in `tutorial10_controllers.ipynb`.

In this tutorial, we simulate an initially perturbed single lane ring road. We witness in simulation that as time advances, the initial perturbations do not dissipate, but instead propagate and expand until vehicles are forced to periodically stop and accelerate. For more information on this behavior, we refer the reader to the following article [1].

## 1. Components of a Simulation
All simulations, both in the presence and absence of RL, require two components: a *network*, and an *environment*. Networks describe the features of the transportation network used in simulation. This includes the positions and properties of nodes and edges constituting the lanes and junctions, as well as properties of the vehicles, traffic lights, inflows, etc. in the network. Environments, on the other hand, initialize, reset, and advance simulations, and act the primary interface between the reinforcement learning algorithm and the network. Moreover, custom environments may be used to modify the dynamical features of an network.

## 2. Setting up a Network
Flow contains a plethora of pre-designed networks used to replicate highways, intersections, and merges in both closed and open settings. All these networks are located in flow/networks. In order to recreate a ring road network, we begin by importing the network `RingNetwork`.

In [1]:
from flow.networks.ring import RingNetwork

This network, as well as all other network in Flow, is parametrized by the following arguments: 
* name
* vehicles
* net_params
* initial_config
* traffic_lights

These parameters allow a single network to be recycled for a multitude of different network settings. For example, `RingNetwork` may be used to create ring roads of variable length with a variable number of lanes and vehicles.

### 2.1 Name
The `name` argument is a string variable depicting the name of the network. This has no effect on the type of network created.

In [2]:
name = "ring_example"

### 2.2 VehicleParams
The `VehicleParams` class stores state information on all vehicles in the network. This class is used to identify the dynamical behavior of a vehicle and whether it is controlled by a reinforcement learning agent. Morover, information pertaining to the observations and reward function can be collected from various get methods within this class.

The initial configuration of this class describes the number of vehicles in the network at the start of every simulation, as well as the properties of these vehicles. We begin by creating an empty `VehicleParams` object.

In [3]:
from flow.core.params import VehicleParams

vehicles = VehicleParams()

Once this object is created, vehicles may be introduced using the `add` method. This method specifies the types and quantities of vehicles at the start of a simulation rollout. For a description of the various arguements associated with the `add` method, we refer the reader to the following documentation (reference readthedocs).

When adding vehicles, their dynamical behaviors may be specified either by the simulator (default), or by user-generated models. For longitudinal (acceleration) dynamics, several prominent car-following models are implemented in Flow. For this example, the acceleration behavior of all vehicles will be defined by the Intelligent Driver Model (IDM) [2].

In [4]:
from flow.controllers.car_following_models import IDMController

While Flow does support the use of custom routers is sumo, this feature is not currently available when using Aimsun. Vehciles, instead, will continuously route to the next available edge. Accordingly, we will not specify a routing controller.

Finally, we add 22 vehicles of type "human" with the above acceleration and routing behavior into the `Vehicles` class.

In [5]:
vehicles.add("human",
             acceleration_controller=(IDMController, {}),
             num_vehicles=22)

### 2.3 NetParams

`NetParams` are network-specific parameters used to define the shape and properties of a network. Unlike most other parameters, `NetParams` may vary drastically depending on the specific network configuration, and accordingly most of its parameters are stored in `additional_params`. In order to determine which `additional_params` variables may be needed for a specific network, we refer to the `ADDITIONAL_NET_PARAMS` variable located in the network file.

In [6]:
from flow.networks.ring import ADDITIONAL_NET_PARAMS

print(ADDITIONAL_NET_PARAMS)

{'length': 230, 'lanes': 1, 'speed_limit': 30, 'resolution': 40}


Importing the `ADDITIONAL_NET_PARAMS` dict from the ring road network, we see that the required parameters are:

* **length**: length of the ring road
* **lanes**: number of lanes
* **speed**: speed limit for all edges
* **resolution**: resolution of the curves on the ring. Setting this value to 1 converts the ring to a diamond.


At times, other inputs may be needed from `NetParams` to recreate proper network features/behavior. These requirements can be founded in the network's documentation. For the ring road, no attributes are needed aside from the `additional_params` terms. Furthermore, for this tutorial, we use the network's default parameters when creating the `NetParams` object.

In [7]:
from flow.core.params import NetParams

net_params = NetParams(additional_params=ADDITIONAL_NET_PARAMS)

### 2.4 InitialConfig

`InitialConfig` specifies parameters that affect the positioning of vehicle in the network at the start of a simulation. These parameters can be used to limit the edges and number of lanes vehicles originally occupy, and provide a means of adding randomness to the starting positions of vehicles. In order to introduce a small initial disturbance to the system of vehicles in the network, we set the `perturbation` term in `InitialConfig` to 1m.

In [8]:
from flow.core.params import InitialConfig

initial_config = InitialConfig(spacing="uniform", perturbation=1)

### 2.5 TrafficLightParams

`TrafficLightParams` are used to desribe the positions and types of traffic lights in the network. These inputs are outside the scope of this tutorial, and instead are covered in `tutorial10_traffic_lights.ipynb`. For our example, we create an empty `TrafficLightParams` object, thereby ensuring that none are placed on any nodes.

In [9]:
from flow.core.params import TrafficLightParams

traffic_lights = TrafficLightParams()

### 2.6 Aimsun Template

While the aimsun template is not a parameter within the network class, it is a significant component of the network that will be generated. The default template in Flow is located in:

```
/path/to/flow/flow/utils/aimsun/Aimsun_Flow.ang
```

This template is further modified to create the network we will see towards the end of this tutorial.

If you were to double click on this template, a window similar to the one below will appear. From here, you can modify any network or simulation parameters you would like that are not currently available within Flow. Being that the incorporation of Aimsun into Flow is still in its early stages, not many features are currently available, so modifying this template may be of use at times.

<img src="img/aimsun_template.png" width="1000">

## 3. Setting up an Environment

Several envionrments in Flow exist to train autonomous agents of different forms (e.g. autonomous vehicles, traffic lights) to perform a variety of different tasks. These environments are often network or task specific; however, some can be deployed on an ambiguous set of networks as well. One such environment, `AccelEnv`, may be used to train a variable number of vehicles in a fully observable network with a *static* number of vehicles.

In [10]:
from flow.envs.ring.accel import AccelEnv

Although we will not be training any autonomous agents in this tutorial, the use of an environment allows us to view the cumulative reward simulation rollouts receive in the absence of autonomy.

Envrionments in Flow are parametrized by several components, including the following attributes:
* `sim_params`
* `env_params`
* `network`
* `net_params`
* `initial_config`
* `network`
* `simulator`

where `sim_params`, `env_params`, and `network` the are primary parameters of an environment. For the full list of attributes, please check `class Env` in `flow/envs/base.py`.

Aimsun envrionments in Flow are parametrized by three components:
* `AimsunParams`
* `EnvParams`
* `Network`

### 3.1 AimsunParams
`AimsunParams` specifies simulation-specific variables. These variables include the length a simulation step (in seconds) and whether to render the GUI when running the experiment. For this example, we set `render` within the simulation params to be True in order for vehicles to appear on the GUI during the simulation.

In [11]:
from flow.core.params import AimsunParams

sim_params = AimsunParams(render=True)

Note that if `render` were set to False the GUI would still turn on, but no vehicles or traffic lights would be rendered. This significantly improves runtimes. It is also worth mentioning that, being that the incorporation of Aimsun into Flow is still in beta stage, only the `render`, `sim_step`, and `emission_path` parameters within `AimsunParams` currently function. If you would like to update the simulation step, for instance, this must be done directly from the default template (see section 2.6).

### 3.2 EnvParams

`EnvParams` specify environment and experiment-specific parameters that either affect the training process or the dynamics of various components within the network. Much like `NetParams`, the attributes associated with this parameter are mostly environment specific, and can be found in the environment's `ADDITIONAL_ENV_PARAMS` dictionary.

In [12]:
from flow.envs.ring.accel import ADDITIONAL_ENV_PARAMS

print(ADDITIONAL_ENV_PARAMS)

{'max_accel': 3, 'max_decel': 3, 'target_velocity': 10, 'sort_vehicles': False}


Importing the `ADDITIONAL_ENV_PARAMS` variable, we see that it consists of only one entry, "target_velocity", which is used when computing the reward function associated with the environment. We use this default value when generating the `EnvParams` object.

In [13]:
from flow.core.params import EnvParams

env_params = EnvParams(additional_params=ADDITIONAL_ENV_PARAMS)

## 4. Setting up and Running the Experiment

Once the inputs to the network and environment classes are ready, we are ready to set up a `Experiment` object.

In [14]:
from flow.core.experiment import Experiment

These objects may be used to simulate rollouts in the absence of reinforcement learning agents, as well as acquire behaviors and rewards that may be used as a baseline with which to compare the performance of the learning agent. In this case, we choose to run our experiment for one rollout consisting of 3000 steps (300 s). Note that, unlike in tutorial 1, here we use the Aimsun simulator when running the ring road experiment by setting the `simulator` attribute of the environment as "aimun". The default simulator for all environments in Flow is sumo.

In order to start the simulation, run the below cell. A new Aimsun GUI will open and prompt you to save the file. Save this file anywhere (so long as you do not overwrite the default template), and once you have done so, the simulation will execute.

In [15]:
flow_params = dict(
    exp_tag='test_network',
    env_name=AccelEnv,
    network=RingNetwork,
    simulator='aimsun',
    sim=sim_params,
    env=env_params,
    net=net_params,
    veh=vehicles,
    initial=initial_config,
    tls=traffic_lights
)

# number of time steps
flow_params['env'].horizon = 3000
exp = Experiment(flow_params)

# run the Aimsun simulation
_ = exp.run(1)

<class 'flow.networks.ring.RingNetwork'>


TypeError: Can't instantiate abstract class AimsunKernelVehicle with abstract methods get_2d_position, get_accel, get_realized_accel, get_road_grade, update_accel

If you encounter an error of the form `ImportError: No module named flow` within Aimsun, make sure that you have correctly setup the `aimsun_flow` environment ([installation instructions](https://flow.readthedocs.io/en/latest/flow_setup.html#installing-aimsun)) and that you are running this notebook inside the `flow` environment (for more information on this, see the [note in README.md](https://github.com/flow-project/flow/blob/master/tutorials/README.md#tutorials)).

## 5. Modifying the Simulation
This tutorial has walked you through running a single lane ring road experiment in Flow. As we have mentioned before, these simulations are highly parametrizable. This allows us to try different representations of the task. For example, what happens if no initial perturbations are introduced to the system of homogenous human-driven vehicles?

```
initial_config = InitialConfig()
```

In addition, how does the task change in the presence of multiple lanes where vehicles can overtake one another?

```
net_params = NetParams(
    additional_params={
        'length': 230, 
        'lanes': 2, 
        'speed_limit': 30, 
        'resolution': 40
    }
)
```

Feel free to experiment with all these problems and more!

## Bibliography
[1] Sugiyama, Yuki, et al. "Traffic jams without bottlenecks—experimental evidence for the physical mechanism of the formation of a jam." New journal of physics 10.3 (2008): 033001.

[2] Treiber, Martin, Ansgar Hennecke, and Dirk Helbing. "Congested traffic states in empirical observations and microscopic simulations." Physical review E 62.2 (2000): 1805.